# Análise Eleitoral

In [27]:
from pathlib import Path

root = Path().resolve().parent #.parent attr caso o arquivo nao esteja na raiz do projeto

In [28]:
# Adicionando src/ ao projeto
import sys
import os

src_path = os.path.join(root, 'src')
sys.path.append(src_path)

In [29]:
# Importando configurações do projeto
import yaml

config_path = root.joinpath('config/config.yaml')

def load_config(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)
        
config = load_config(config_path)

### 0) Configuração dos bancos a serem utilizados

In [30]:
import pandas as pd
import tse

In [31]:
CANDIDATO = 'EDUARDO PAES'
ESTADO = 'rj'.lower()
ANO = '2024'

In [32]:
database_dir = config['paths']['sql_data']
database_1 = f'{database_dir}/{ESTADO}_{ANO}.sql'
database_2 = f'{database_dir}/eleicoes_{ANO}.sql'

In [33]:
analise = tse.TseAnalysis(
    database={
        'path': database_1,
        'alias': f'{ESTADO}'
    })

In [34]:
analise.attach_database(
    attach= [
        {
            'path': database_2,
            'alias': 'geral'
        }
    ])

Bancos carregados:
0) main
-path: d:\projects\tse-data\data\databases\rj_2024.sql
--tables: 
-- resultados
-- eleitorado
-----
2) rj
-path: d:\projects\tse-data\data\databases\rj_2024.sql
--tables: 
-- resultados
-- eleitorado
-----
3) geral
-path: d:\projects\tse-data\data\databases\eleicoes_2024.sql
--tables: 
-- locais
-----


In [35]:
analise.create_index(
    table='resultados',
    columns=['NM_VOTAVEL', 'NR_LOCAL_VOTACAO', 'NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO']
)
analise.create_index(
    table='eleitorado',
    columns=['NR_LOCAL_VOTACAO', 'NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO']
)

'Indices criados com sucesso'

---

### 1) Criação de tabela com perfil de eleitorado por seção

In [36]:
# a função generate_stats_query foi escrita para gerar uma query específica para a tabela {ESTADO}. eleitorado
# ela analisa os valores distintos de cada coluna argumento e produz uma query que soma a quantidade de aparições para cada caso
q = analise.generate_stats_query(
    alias=f'{ESTADO}',
    table='eleitorado',
    cols=[
        'DS_GRAU_ESCOLARIDADE',
        'DS_RACA_COR',
        'DS_FAIXA_ETARIA',
        'DS_GENERO'
    ]
)

In [38]:
eleitorado_secao = pd.read_sql(q, analise.conn)

In [39]:
eleitorado_secao.head(2) # este DataFrame expressa a quantidade de eleitores por seçao divido em categorias

,municipio,local,zona,secao,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_ensino_fundamental_incompleto,ds_grau_escolaridade_ensino_fundamental_completo,ds_grau_escolaridade_lê_e_escreve,ds_grau_escolaridade_ensino_médio_completo,...,ds_faixa_etaria_85_a_89_anos,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_16_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,ds_genero_não_informado
0,58009,1015,255,23,61,23,114,37,42,91,...,10,4,0,1,7,0,0,175,211,0
1,58009,1015,255,24,79,29,99,26,48,83,...,2,0,1,0,10,1,0,183,203,0


In [40]:
q_2 = f"""
    SELECT
        CD_MUNICIPIO as municipio,
        NR_LOCAL_VOTACAO as local,
        NR_ZONA as zona,
        NR_SECAO as secao,
        QT_ELEITOR_SECAO as eleitores_totais
    FROM
        geral.locais
    WHERE
        SG_UF = '{ESTADO.upper()}'
"""

In [41]:
eleitores_secao = pd.read_sql(q_2, analise.conn)

In [42]:
eleitores_secao.head(2) # complementa o DataFrame anterior com a quantidade total de eleitores na seção

,municipio,local,zona,secao,eleitores_totais
0,60011,1252,185,136,406
1,60011,1341,14,164,446


In [43]:
eleitorado_secao = pd.merge(
    eleitorado_secao,
    eleitores_secao,
    how='inner',
    on=[
        'municipio',
        'local',
        'zona',
        'secao'
    ]
)

In [44]:
eleitorado_secao.head(2) # resultado obtido

,municipio,local,zona,secao,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_ensino_fundamental_incompleto,ds_grau_escolaridade_ensino_fundamental_completo,ds_grau_escolaridade_lê_e_escreve,ds_grau_escolaridade_ensino_médio_completo,...,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_16_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,ds_genero_não_informado,eleitores_totais
0,58009,1015,255,23,61,23,114,37,42,91,...,4,0,1,7,0,0,175,211,0,386
1,58009,1015,255,24,79,29,99,26,48,83,...,0,1,0,10,1,0,183,203,0,386


---

### 2) Cria tabela com apuração dos resultados por seção de determinado candidato

In [45]:
query = f"""
    SELECT
        NM_VOTAVEL as candidato,
        CD_MUNICIPIO as municipio,
        NR_LOCAL_VOTACAO as local,
        NR_ZONA as zona,
        NR_SECAO as secao,
        NR_TURNO as turno,
        QT_VOTOS as votos_recebidos
    FROM
        {ESTADO}.resultados
    WHERE
        NM_VOTAVEL LIKE '{'%' + CANDIDATO.replace(' ', '%') + '%'}'
"""

In [46]:
resultado_secao = pd.read_sql(query, analise.conn)

In [47]:
resultado_secao.head(2)

,candidato,municipio,local,zona,secao,turno,votos_recebidos
0,EDUARDO DA COSTA PAES,60011,1317,245,254,1,170
1,EDUARDO DA COSTA PAES,60011,1341,123,262,1,155


In [48]:
resultado_perfil = pd.merge(
    resultado_secao,
    eleitorado_secao,
    how='inner',
    on=['municipio', 'local', 'zona', 'secao']
)

In [49]:
resultado_perfil.head(2)

,candidato,municipio,local,zona,secao,turno,votos_recebidos,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_ensino_fundamental_incompleto,...,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_16_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,ds_genero_não_informado,eleitores_totais
0,EDUARDO DA COSTA PAES,60011,1317,245,254,1,170,90,38,60,...,4,8,10,0,1,0,263,194,2,459
1,EDUARDO DA COSTA PAES,60011,1341,123,262,1,155,70,18,100,...,7,10,11,0,0,0,239,186,0,425


In [50]:
resultado_perfil['percentual_votos'] = resultado_perfil['votos_recebidos']/resultado_perfil['eleitores_totais']

In [51]:
output_dir = config['paths']['processed_data']
nome = resultado_perfil['candidato'][0].replace(' ', '_')

In [52]:
resultado_perfil.to_csv(f'{output_dir}/{nome}.csv')